In [1]:
import pandas as pd
import numpy as np

IMPORT DATASET


In [2]:
data = pd.read_csv("./train.csv", index_col='PassengerId')
data['Sex'] = data['Sex'].replace({'male':1, 'female':0})
data.corr(numeric_only=True).abs()[['Survived']]

C:\Users\MSI VN\AppData\Local\Temp\ipykernel_21172\1081518529.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Sex'] = data['Sex'].replace({'male':1, 'female':0})


,Survived
Survived,1.000000
Pclass,0.338481
Sex,0.543351
Age,0.077221
SibSp,0.035322
Parch,0.081629
Fare,0.257307


In [26]:
s = data.select_dtypes(include = ['object'])

In [28]:
s

,Name,Ticket,Cabin,Embarked
PassengerId,,,,
1,"Braund, Mr. Owen Harris",A/5 21171,U,S
2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",PC 17599,C,C
3,"Heikkinen, Miss. Laina",STON/O2. 3101282,U,S
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",113803,C,S
5,"Allen, Mr. William Henry",373450,U,S
...,...,...,...,...
887,"Montvila, Rev. Juozas",211536,U,S
888,"Graham, Miss. Margaret Edith",112053,B,S
889,"Johnston, Miss. Catherine Helen ""Carrie""",W./C. 6607,U,S


In [3]:
data['Cabin'].nunique()

147

In [4]:
features = ['Pclass', 'Sex', 'Age', 'SibSp','Cabin','Fare', 'Embarked']

PROCESSING FARE AND CABIN 

In [5]:
data['Fare'] = np.log1p(data['Fare'])
data['Cabin'] = data['Cabin'].str[0].fillna("U")
data[['Cabin','Fare']].head()
train_feature = data[features]
train_feature.head()

,Pclass,Sex,Age,SibSp,Cabin,Fare,Embarked
PassengerId,,,,,,,
1,3,1,22.0,1,U,2.110213,S
2,1,0,38.0,1,C,4.280593,C
3,3,0,26.0,0,U,2.188856,S
4,1,0,35.0,1,C,3.990834,S
5,3,1,35.0,0,U,2.202765,S


PROCESSING EMBARKED AND SEX

In [6]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
train_feature['Embarked'] = LE.fit_transform(train_feature['Embarked'])
train_feature['Sex'] = LE.fit_transform(train_feature['Sex'])


C:\Users\MSI VN\AppData\Local\Temp\ipykernel_21172\2945137669.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_feature['Embarked'] = LE.fit_transform(train_feature['Embarked'])
C:\Users\MSI VN\AppData\Local\Temp\ipykernel_21172\2945137669.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_feature['Sex'] = LE.fit_transform(train_feature['Sex'])


SPLIT TRAINING SET


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_feature, data['Survived'], train_size=0.8)

PREPROCESSING : IMPUTING


In [8]:
from sklearn.impute import SimpleImputer
SI = SimpleImputer(strategy='mean')
x_train['Age'] = SI.fit_transform(x_train[['Age']])
x_test['Age'] = SI.transform(x_test[['Age']])

NHÓM TUỔI


In [9]:
x_train['Age'] = pd.cut(x_train['Age'], bins=[0,12,18,35,60,80], labels=["Child","Teen","YoungAdult","Adult","Senior"])
x_test['Age'] = pd.cut(x_test['Age'], bins=[0,12,18,35,60,80], labels=["Child","Teen","YoungAdult","Adult","Senior"])

In [10]:
type(x_train)

pandas.core.frame.DataFrame

ONEHOT CHO AGE, CABIN

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
CT = ColumnTransformer(transformers = [('encode', OneHotEncoder(), ['Age','Cabin'])], remainder = 'passthrough')
# Biến đổi
train_transformed = CT.fit_transform(x_train)
test_transformed = CT.transform(x_test)
# Lấy tên cột mới
new_cols = CT.get_feature_names_out()
# Tạo lại DataFrame
x_train = pd.DataFrame(train_transformed, columns=new_cols, index = x_train.index)
x_test = pd.DataFrame(test_transformed, columns=new_cols, index = x_test.index)

TRAINING MODEL


In [12]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
RFC.fit(x_train, y_train)
y_pred = RFC.predict(x_test)
result = pd.DataFrame({'Prediction' : y_pred,
                       'Reality' : y_test})
result.head()

,Prediction,Reality
PassengerId,,
214,0,0
512,0,0
52,0,0
4,1,1
666,0,0


ACCURACY CHECKING 

In [13]:
from sklearn.metrics import accuracy_score 
acc = accuracy_score(y_test, y_pred)
print(f'tỉ lệ dự đoán kết quả sống chết của kĩ sư chuyên ngành tự động hóa phân nhánh quản lý dữ liệu thiết bị: {acc*100}%')

tỉ lệ dự đoán kết quả sống chết của kĩ sư chuyên ngành tự động hóa phân nhánh quản lý dữ liệu thiết bị: 78.77094972067039%
